# US yield curve study, source FED

In [ ]:
import pandas as pd
import numpy as np
import os
from utils import *
from datetime import datetime
from sklearn.decomposition import PCA

In [ ]:
nominal_yld_curve = pd.read_csv("feds200628.csv", skiprows=8)
tips_yld_curve    = pd.read_csv("feds200805.csv", skiprows=18)
nominal_yld_curve[:3]

In [ ]:
# parse date
def parse_date(df):
    df["Date"] = df["Date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
    df.set_index("Date", inplace=True)
parse_date(nominal_yld_curve)
parse_date(tips_yld_curve)

In [ ]:
cols_nominal = [(x, int(x[-2:])/12.0) for x in (nominal_yld_curve.columns) if "SVEN1F" in x]

cols_nominal = []
cols_nominal = cols_nominal + [(x, int(x[-2:])) for x in (nominal_yld_curve.columns) if "SVENF" in x]
dict_cols_nominal = {}
for d in cols_nominal: 
    dict_cols_nominal[d[0]] = d[1]

In [ ]:
print("on the run bond yield")
df2plot = nominal_yld_curve[dict_cols_nominal.keys()]["2006-01-01":].resample("1m").last()
plot_plotly(df2plot, width=1500)

print("yield curve")
df2plot = nominal_yld_curve[dict_cols_nominal.keys()].rename(columns=dict_cols_nominal)['2005-01-01':'2022-12-31'].resample('6m').last()
plot_plotly(df2plot.T[:20], width=1500)

print("key slope spread")
pairs = [(10, 2), (7, 2), (10, 5), (20, 10), (12, 10), (10, 7)]
df_spread = nominal_yld_curve[dict_cols_nominal.keys()].rename(columns=dict_cols_nominal)['1995-01-01':'2022-12-31']
dict_spread = dict()
for p in pairs:
    p1, p2 = max(p), min(p)
    dict_spread["spread_{}_{}".format(p1, p2)] = df_spread[p1] - df_spread[p2]
df_spread = pd.DataFrame(dict_spread)
df_spread['level_0'] = 0
plot_plotly(df_spread, width=1500)

In [ ]:
print("PCA decomposition")
n_comp = 5
df_pca = nominal_yld_curve[dict_cols_nominal.keys()].rename(columns=dict_cols_nominal)['1996-01-03':'2022-03-31']
df_pca = df_pca[df_pca.isna().sum(axis=1)==0]
pca = PCA(n_components=n_comp, svd_solver='full')
pca.fit(df_pca.values)

rename_dict = {}
for k in range(n_comp):
    rename_dict[k] = str(k)

pca_fit = pca.fit_transform(df_pca.values)
pca_fit = pd.DataFrame(index=df_pca.index, data = pca_fit)
pca_fit =( pca_fit - pca_fit.mean() ) / pca_fit.std()
plot_plotly((pca_fit).rename(columns=rename_dict), width=1500)